<a href="https://colab.research.google.com/github/LeToyek/2141720085-machine-learning-2023/blob/main/praktikum_2(Tugas_Praktikum_Included).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
import numpy as np
import os
import time

In [2]:
path_to_file = tf.keras.utils.get_file('shakespeare.txt', 'https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt')

1115394/1115394 [==============================] - 0s 0us/step


In [3]:
# Read, then decode for py2 compat.
text = open(path_to_file, 'rb').read().decode(encoding='utf-8')
# length of text is the number of characters in it
print(f'Length of text: {len(text)} characters')

Length of text: 1115394 characters


In [4]:
# Take a look at the first 250 characters in text
print(text[:250])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.



In [5]:
# The unique characters in the file
vocab = sorted(set(text))
print(f'{len(vocab)} unique characters')

65 unique characters


In [6]:
example_texts = ['abcdefg', 'xyz']
chars = tf.strings.unicode_split(example_texts, input_encoding='UTF-8')
chars

<tf.RaggedTensor [[b'a', b'b', b'c', b'd', b'e', b'f', b'g'], [b'x', b'y', b'z']]>

In [7]:
ids_from_chars=tf.keras.layers.StringLookup(vocabulary=list(vocab),mask_token=None)

In [8]:
ids = ids_from_chars(chars)
ids

<tf.RaggedTensor [[40, 41, 42, 43, 44, 45, 46], [63, 64, 65]]>

In [9]:
chars_from_ids = tf.keras.layers.StringLookup(
    vocabulary=ids_from_chars.get_vocabulary(), invert=True, mask_token=None)

In [10]:
chars = chars_from_ids(ids)
chars

<tf.RaggedTensor [[b'a', b'b', b'c', b'd', b'e', b'f', b'g'], [b'x', b'y', b'z']]>

In [11]:
tf.strings.reduce_join(chars,axis=-1).numpy()

array([b'abcdefg', b'xyz'], dtype=object)

In [12]:
def text_from_ids(ids):
    return tf.strings.reduce_join(chars_from_ids(ids), axis=-1)

In [13]:
all_ids = ids_from_chars(tf.strings.unicode_split(text, 'UTF-8'))
all_ids

<tf.Tensor: shape=(1115394,), dtype=int64, numpy=array([19, 48, 57, ..., 46,  9,  1])>

In [14]:
ids_dataset = tf.data.Dataset.from_tensor_slices(all_ids)

In [15]:
for ids in ids_dataset.take(10):
    print(chars_from_ids(ids).numpy().decode('utf-8'))

F
i
r
s
t
 
C
i
t
i


In [16]:
seq_length=100

In [17]:
sequences = ids_dataset.batch(seq_length+1, drop_remainder=True)

for seq in sequences.take(1):
  print(chars_from_ids(seq))

tf.Tensor(
[b'F' b'i' b'r' b's' b't' b' ' b'C' b'i' b't' b'i' b'z' b'e' b'n' b':'
 b'\n' b'B' b'e' b'f' b'o' b'r' b'e' b' ' b'w' b'e' b' ' b'p' b'r' b'o'
 b'c' b'e' b'e' b'd' b' ' b'a' b'n' b'y' b' ' b'f' b'u' b'r' b't' b'h'
 b'e' b'r' b',' b' ' b'h' b'e' b'a' b'r' b' ' b'm' b'e' b' ' b's' b'p'
 b'e' b'a' b'k' b'.' b'\n' b'\n' b'A' b'l' b'l' b':' b'\n' b'S' b'p' b'e'
 b'a' b'k' b',' b' ' b's' b'p' b'e' b'a' b'k' b'.' b'\n' b'\n' b'F' b'i'
 b'r' b's' b't' b' ' b'C' b'i' b't' b'i' b'z' b'e' b'n' b':' b'\n' b'Y'
 b'o' b'u' b' '], shape=(101,), dtype=string)


In [18]:
for seq in sequences.take(5):
    print(text_from_ids(seq).numpy())

b'First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou '
b'are all resolved rather to die than to famish?\n\nAll:\nResolved. resolved.\n\nFirst Citizen:\nFirst, you k'
b"now Caius Marcius is chief enemy to the people.\n\nAll:\nWe know't, we know't.\n\nFirst Citizen:\nLet us ki"
b"ll him, and we'll have corn at our own price.\nIs't a verdict?\n\nAll:\nNo more talking on't; let it be d"
b'one: away, away!\n\nSecond Citizen:\nOne word, good citizens.\n\nFirst Citizen:\nWe are accounted poor citi'


In [19]:
def split_input_target(sequence):
 input_text = sequence[:-1]
 target_text = sequence[1:]
 return input_text, target_text

In [20]:
split_input_target(list("Tensorflow"))

(['T', 'e', 'n', 's', 'o', 'r', 'f', 'l', 'o'],
 ['e', 'n', 's', 'o', 'r', 'f', 'l', 'o', 'w'])

In [21]:
dataset = sequences.map(split_input_target)

In [22]:
for input_example, target_example in dataset.take(1):
 print("Input :", text_from_ids(input_example).numpy())
 print("Target:", text_from_ids(target_example).numpy())

Input : b'First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou'
Target: b'irst Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou '


In [23]:
# Batch size
BATCH_SIZE = 64

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 10000

dataset = (
    dataset
    .shuffle(BUFFER_SIZE)
    .batch(BATCH_SIZE, drop_remainder=True)
    .prefetch(tf.data.experimental.AUTOTUNE))

dataset

<_PrefetchDataset element_spec=(TensorSpec(shape=(64, 100), dtype=tf.int64, name=None), TensorSpec(shape=(64, 100), dtype=tf.int64, name=None))>

In [24]:
# Length of the vocabulary in StringLookup Layer
vocab_size = len(ids_from_chars.get_vocabulary())

# The embedding dimension
embedding_dim = 256

# Number of RNN units
rnn_units = 1024

In [25]:
class MyModel(tf.keras.Model):
  def __init__(self, vocab_size, embedding_dim, rnn_units):
    super().__init__(self)
    self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
    self.gru = tf.keras.layers.GRU(rnn_units,
                                   return_sequences=True,
                                   return_state=True)
    self.dense = tf.keras.layers.Dense(vocab_size)

  def call(self, inputs, states=None, return_state=False, training=False):
    x = inputs
    x = self.embedding(x, training=training)
    if states is None:
      states = self.gru.get_initial_state(x)
    x, states = self.gru(x, initial_state=states, training=training)
    x = self.dense(x, training=training)

    if return_state:
      return x, states
    else:
      return x

In [26]:
model = MyModel(
    vocab_size=vocab_size,
    embedding_dim=embedding_dim,
    rnn_units=rnn_units)

In [27]:
for input_example_batch, target_example_batch in dataset.take(1):
    example_batch_predictions = model(input_example_batch)
    print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

(64, 100, 66) # (batch_size, sequence_length, vocab_size)


In [28]:
model.summary()

Model: "my_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       multiple                  16896     
                                                                 
 gru (GRU)                   multiple                  3938304   
                                                                 
 dense (Dense)               multiple                  67650     
                                                                 
Total params: 4022850 (15.35 MB)
Trainable params: 4022850 (15.35 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [29]:
sampled_indices=tf.random.categorical(example_batch_predictions[0],num_samples=1)
sampled_indices=tf.squeeze(sampled_indices,axis=-1).numpy()

In [30]:
sampled_indices

array([ 9,  7, 57, 33, 10, 60, 19,  7, 54, 22, 54, 49, 63, 50,  3, 38, 13,
       49,  7,  7, 36, 61, 22, 63, 60, 47, 28,  0, 39, 16, 16, 43, 30, 39,
       43,  0, 30, 39, 44, 39, 17, 12, 12, 11, 46, 49, 15,  8, 17, 38,  5,
       44, 45, 52, 37, 45,  6, 15, 11, 10,  5, 36, 19, 35, 48, 63,  2, 41,
       37, 36,  9, 14, 50, 44, 16,  0, 41, 23, 29, 62, 63, 19, 21, 31, 28,
       31, 19,  1, 21, 58, 25,  4, 23,  5, 41, 11,  2, 10,  0, 13])

In [31]:
print("Input:\n", text_from_ids(input_example_batch[0]).numpy())
print()
print("Next Char Predictions:\n", text_from_ids(sampled_indices).numpy())

Input:
 b"\nCome hither, England's hope.\nIf secret powers\nSuggest but truth to my divining thoughts,\nThis prett"

Next Char Predictions:
 b".,rT3uF,oIojxk!Y?j,,WvIxuhO[UNK]ZCCdQZd[UNK]QZeZD;;:gjB-DY&efmXf'B:3&WFVix bXW.AkeC[UNK]bJPwxFHRORF\nHsL$J&b: 3[UNK]?"


In [32]:
loss = tf.losses.SparseCategoricalCrossentropy(from_logits=True)

In [33]:
example_batch_mean_loss = loss(target_example_batch, example_batch_predictions)
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)")
print("Mean loss:        ", example_batch_mean_loss)

Prediction shape:  (64, 100, 66)  # (batch_size, sequence_length, vocab_size)
Mean loss:         tf.Tensor(4.1913357, shape=(), dtype=float32)


In [34]:
tf.exp(example_batch_mean_loss).numpy()

66.11103

In [35]:
model.compile(optimizer='adam',loss=loss)

In [36]:
# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

In [37]:
EPOCHS=20

In [38]:
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])


Epoch 1/20
172/172 [==============================] - 20s 62ms/step - loss: 2.7300
Epoch 2/20
172/172 [==============================] - 14s 59ms/step - loss: 1.9984
Epoch 3/20
172/172 [==============================] - 11s 57ms/step - loss: 1.7234
Epoch 4/20
172/172 [==============================] - 12s 60ms/step - loss: 1.5617
Epoch 5/20
172/172 [==============================] - 12s 62ms/step - loss: 1.4606
Epoch 6/20
172/172 [==============================] - 12s 62ms/step - loss: 1.3908
Epoch 7/20
172/172 [==============================] - 13s 65ms/step - loss: 1.3381
Epoch 8/20
172/172 [==============================] - 12s 61ms/step - loss: 1.2931
Epoch 9/20
172/172 [==============================] - 12s 61ms/step - loss: 1.2518
Epoch 10/20
172/172 [==============================] - 12s 61ms/step - loss: 1.2123
Epoch 11/20
172/172 [==============================] - 12s 61ms/step - loss: 1.1733
Epoch 12/20
172/172 [==============================] - 12s 61ms/step - loss: 1.1322
E

In [39]:
class OneStep(tf.keras.Model):
  def __init__(self, model, chars_from_ids, ids_from_chars, temperature=1.0):
    super().__init__()
    self.temperature = temperature
    self.model = model
    self.chars_from_ids = chars_from_ids
    self.ids_from_chars = ids_from_chars

    # Create a mask to prevent "[UNK]" from being generated.
    skip_ids = self.ids_from_chars(['[UNK]'])[:, None]
    sparse_mask = tf.SparseTensor(
        # Put a -inf at each bad index.
        values=[-float('inf')]*len(skip_ids),
        indices=skip_ids,
        # Match the shape to the vocabulary
        dense_shape=[len(ids_from_chars.get_vocabulary())])
    self.prediction_mask = tf.sparse.to_dense(sparse_mask)

  @tf.function
  def generate_one_step(self, inputs, states=None):
    # Convert strings to token IDs.
    input_chars = tf.strings.unicode_split(inputs, 'UTF-8')
    input_ids = self.ids_from_chars(input_chars).to_tensor()

    # Run the model.
    # predicted_logits.shape is [batch, char, next_char_logits]
    predicted_logits, states = self.model(inputs=input_ids, states=states,
                                          return_state=True)
    # Only use the last prediction.
    predicted_logits = predicted_logits[:, -1, :]
    predicted_logits = predicted_logits/self.temperature
    # Apply the prediction mask: prevent "[UNK]" from being generated.
    predicted_logits = predicted_logits + self.prediction_mask

    # Sample the output logits to generate token IDs.
    predicted_ids = tf.random.categorical(predicted_logits, num_samples=1)
    predicted_ids = tf.squeeze(predicted_ids, axis=-1)

    # Convert from token ids to characters
    predicted_chars = self.chars_from_ids(predicted_ids)

    # Return the characters and model state.
    return predicted_chars, states

In [40]:
one_step_model = OneStep(model, chars_from_ids, ids_from_chars)

In [41]:
start = time.time()
states = None
next_char = tf.constant(['ROMEO:'])
result = [next_char]

for n in range(1000):
  next_char, states = one_step_model.generate_one_step(next_char, states=states)
  result.append(next_char)

result = tf.strings.join(result)
end = time.time()
print(result[0].numpy().decode('utf-8'), '\n\n' + '_'*80)
print('\nRun time:', end - start)

ROMEO:
Spakest thou off corrupt, and made
More hate to England, all none falls and very
abo.

SEBASTIAN:
You were known! what knock me with a tail upon these wounds;
Therefore at that title shortly, as hun
receives, and I'll be thy otherwisk.

YORK:
For what, I prophesy, if you crown to him.

QUEEN MARGARET:
I vow'd it. How now, mantles, mistress commands as tender
Than when he speak this business. Worse than wrong'd what satisficius
Mayst thou behold'? on me; Buckingham,
Bid him lead appeareth in this place:
This may pay I no longer, that fur is there;
her face impeach'd, and it becomes
My talk appearable: and so attend on him.
Making the downfall of his flood and his lab
Timush'd, and bay stay and speak to thee rock.

BAPOT:
I do.

HERMIONE:
I pray you, sir, I shall resolve pude men'd
until werk your case out, did read him dead;
And who such wear oppress of the eight,
We'll put you power to bring Edward
Thou poin'd upon the shore, to her my heart
To help him of your youth-be to dinne

In [42]:
start = time.time()
states = None
next_char = tf.constant(['ROMEO:', 'ROMEO:', 'ROMEO:', 'ROMEO:', 'ROMEO:'])
result = [next_char]

for n in range(1000):
  next_char, states = one_step_model.generate_one_step(next_char, states=states)
  result.append(next_char)

result = tf.strings.join(result)
end = time.time()
print(result, '\n\n' + '_'*80)
print('\nRun time:', end - start)

tf.Tensor(
[b"ROMEO:\nGood my lord, be burnt in love and so less, your\nhand, you shall bear a poor with't out: you shall\nheed no more doth make the hardest thicker not\nThe same pardon Rome.'\n\nAUFIDIUS:\nThis your subject, I shall lie with thee\nthat he rose that hust before him, 'fore these Capulet\nUpon this coftar--heart'st and uncle, sir; and therein\nIs my sweet villain; it is ten to order loves.\n\nNORTHUMBERLAND:\nNorthumberland, Isabel, upon that God, I fear 'tis thus:\nHold, then thou hast. O, my holy odd savour\nIs do continue scorn his tatal golder incribents sound\nTo Richmond, and if you had the words resorve\nTo have that plantful pity make I must come all.\n\nESCALUS:\nCommend me to it, let's see thee well:\nThe absolute power, Edward will be farished:\nI'll not be him, I hope to have a dogrement;\nyea, and many fair prosperous enemies.\n\nFRIAR LAURENCE:\nHark, Barrarity lends and flant Aufidius with\nhim to the earth.\n\nSICINIUS:\nThis is your son:\nGood madam, si

In [43]:
tf.saved_model.save(one_step_model, 'one_step')
one_step_reloaded = tf.saved_model.load('one_step')

In [44]:
states = None
next_char = tf.constant(['ROMEO:'])
result = [next_char]

for n in range(100):
  next_char, states = one_step_reloaded.generate_one_step(next_char, states=states)
  result.append(next_char)

print(tf.strings.join(result)[0].numpy().decode("utf-8"))

ROMEO:
Spakest thou to a festive tall yet upon
on him that spoke 'gain that heaven 'tweer and false battle


## Tugas Praktikum

In [45]:
class CustomTraining(MyModel):
  @tf.function
  def train_step(self, inputs):
      inputs, labels = inputs
      with tf.GradientTape() as tape:
          predictions = self(inputs, training=True)
          loss = self.loss(labels, predictions)
      grads = tape.gradient(loss, model.trainable_variables)
      self.optimizer.apply_gradients(zip(grads, model.trainable_variables))

      return {'loss': loss}

In [46]:
model = CustomTraining(
    vocab_size=len(ids_from_chars.get_vocabulary()),
    embedding_dim=embedding_dim,
    rnn_units=rnn_units)

In [47]:
model.compile(optimizer = tf.keras.optimizers.Adam(),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True))

In [48]:
model.fit(dataset, epochs=1)

172/172 [==============================] - 16s 63ms/step - loss: 2.7341


In [49]:
EPOCHS = 10

mean = tf.metrics.Mean()

for epoch in range(EPOCHS):
    start = time.time()

    mean.reset_states()
    for (batch_n, (inp, target)) in enumerate(dataset):
        logs = model.train_step([inp, target])
        mean.update_state(logs['loss'])

        if batch_n % 50 == 0:
            template = f"Epoch {epoch+1} Batch {batch_n} Loss {logs['loss']:.4f}"
            print(template)

    # saving (checkpoint) the model every 5 epochs
    if (epoch + 1) % 5 == 0:
        model.save_weights(checkpoint_prefix.format(epoch=epoch))

    print()
    print(f'Epoch {epoch+1} Loss: {mean.result().numpy():.4f}')
    print(f'Time taken for 1 epoch {time.time() - start:.2f} sec')
    print("_"*80)

model.save_weights(checkpoint_prefix.format(epoch=epoch))

Epoch 1 Batch 0 Loss 2.2088
Epoch 1 Batch 50 Loss 2.0486
Epoch 1 Batch 100 Loss 1.9678
Epoch 1 Batch 150 Loss 1.8613

Epoch 1 Loss: 1.9972
Time taken for 1 epoch 14.02 sec
________________________________________________________________________________
Epoch 2 Batch 0 Loss 1.8173
Epoch 2 Batch 50 Loss 1.7750
Epoch 2 Batch 100 Loss 1.6788
Epoch 2 Batch 150 Loss 1.6394

Epoch 2 Loss: 1.7211
Time taken for 1 epoch 12.18 sec
________________________________________________________________________________
Epoch 3 Batch 0 Loss 1.6365
Epoch 3 Batch 50 Loss 1.5791
Epoch 3 Batch 100 Loss 1.5535
Epoch 3 Batch 150 Loss 1.4924

Epoch 3 Loss: 1.5588
Time taken for 1 epoch 12.18 sec
________________________________________________________________________________
Epoch 4 Batch 0 Loss 1.4974
Epoch 4 Batch 50 Loss 1.4657
Epoch 4 Batch 100 Loss 1.4798
Epoch 4 Batch 150 Loss 1.4210

Epoch 4 Loss: 1.4588
Time taken for 1 epoch 12.05 sec
_____________________________________________________________________

Jawaban


Pada praktikum 2, pendekatan pelatihan yang lebih umum dan sederhana dengan menggunakan metode model.fit dapat mengelola banyak aspek pelatihan, seperti perhitungan loss, gradien, dan pembaruan bobot model. Sementara kode pada tugas praktikum dibuat menjadi lebih detail sehingga parameter yang digunakan tidak hanya terbatas seperti pada function model.fit(). Perbedaan yang terlihat terdapat pada waktu pemrosesan per epoch. Kode pada tugas praktikum dapat memproses lebih cepat dari pada model.fit() dengan loss yang tidak jauh berbeda